# **Predicting Concrete Compressive Strength**

### Motivation:
Concrete is a fundamental material in civil engineering. Its compressive strength is important for ensuring the safety and durability of structures. However, determining this strength accurately is challenging because it depends on a nonlinear function between several factors, including the age of the concrete and the proportions of its components—such as cement, blast furnace slag, fly ash, water, superplasticizer, coarse aggregate, and fine aggregate.
This project aims to apply machine learning techniques to improve the prediction of concrete strength, in order to contribute to better design, quality control, and resource optimization in construction.

### Dataset Explanation:
The dataset is sourced from the UCI Machine Learning Repository and contains 1030 observations with 8 quantitative input variables, and 1 quantitative output variable.
The input variables are: cement, blast furnace slag, fly ash, water, superplasticizer, coarse aggregate, fine aggregate and age. Except for age, the other variables are ingredient quantities in the concrete mixture and measured in *kg in a $m^{3}$* mixture. The age is measured in *days*.
The target, concrete compressive strengt, is measured in *MPa*.

The dataset is available under: https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength

### Data Analytics: Preprocessing
Since there is no missing attribute values according to the information provided by UCI Machine Learning Repository. Data can be directly read from Concrete_data.csv. 

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn

from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
from itertools import combinations
from IPython.display import display, Math

from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, make_scorer, r2_score

from sklearn.model_selection import KFold, cross_validate

%matplotlib inline
plt.style.use('ggplot') 
sns.set_style("darkgrid")
# print numpy arrays with precision 4
np.set_printoptions(precision=4)

In [ ]:
concrete_df = pd.read_csv('./Concrete_data.csv', 
                          sep = ',',
                         decimal = ',',
                         encoding = 'UTF-8')
concrete_df.columns = concrete_df.columns.str.strip()
concrete_df.head()

In [ ]:
print("Original data types：")
print(concrete_df.dtypes)

# object to numeric
for col in concrete_df.columns:
    if concrete_df[col].dtype == 'object':
        concrete_df[col] = pd.to_numeric(concrete_df[col], errors='coerce')


print("\nTransfromed datatypes：")
print(concrete_df.dtypes)

# NaN 
print("\nRows with NaN：", concrete_df.isna().sum().sum())

# Drop NaN
concrete_df = concrete_df.dropna()
concrete_df.head()

In [ ]:
#define all the variables we need for the linear regression with concrete compressive strength as response and all other variables as possible features
label_column = 'Concrete compressive strength(MPa, megapascals)'
feature_columns = [c for c in concrete_df.columns if c != label_column]
X = concrete_df[feature_columns].values
y = concrete_df[label_column].values
p = len(feature_columns)
print("Features are:", feature_columns,
      ", and the response variable is:", label_column)

#### Explanation of relevant variables:

The following table shows the meaning and expected effect of respective variables on the concrete compressive strength. 
| Variable             | Meaning                              | Expected Effect on Strength    |
| -------------------- | ------------------------------------ | ------------------------------ |
| `cement`             | Binder                               | increases strength          |
| `blast_furnace_slag` | Supplementary binder                 | increases strength or neutral                  |
| `fly_ash`            | Supplementary binder                 | increase strength or neutral                  |
| `water`              | Affects workability and hydration    | decreases strength (excess) |
| `superplasticizer`   | Increases workability, reduces water | increase or decrease depending on context  |
| `coarse_aggregate`   | Filler                               | neutral or mild effect         |
| `fine_aggregate`     | Filler                               | neutral or mild effect         |
| `age`                | Curing time                          | increases strength          |


#### Feature scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

#### Distribution of target variable

In [ ]:
sns.histplot(concrete_df[label_column], kde=True)
plt.title("Distribution of Compressive Strength")
plt.show()

The sns.histplot clearly illustrates that the compressive strength data is not uniformly or normally distributed. Instead, it shows a sporadic and spread-out distribution with many unique or rarely occurring strength values, and potentially some concentrations at the extreme ends of the
measured range.
The data distribution is sparse, has a lack of strong central Tendency, so we need to avoid assumption-heavy models (Simple linear regression) and consider robust models (Decision-tree is expected to perform the best).
That doesn’t mean that we don’t try simple linear regression, but maybe this can already explain why linear regression doesn’t perform well.

#### Correlation between relevant variables:

In [ ]:
corr = concrete_df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.show()

#### Variables that influence concrete strength:

According to the result of the heatmap, it can be seen that **cement** has a moderately strong positive correlation with concrete strength with a value of +0.50, which indicates more cement usually leads to stronger concrete. **Superplasticizer** also has a positive effect with a value of +0.37.
**Age** has a positive effect too , because concrete gets stronger over time.
**Water** has a negative correlation with strength of -0.29, which means more water tends to weaken concrete. This observation is consistent with **Abrams' water-to-cement ratio pronouncement**, which emphasizes that increased water relative to cement typically reduces concrete strength.

The other components like fly Ash, Coarse Aggregate and Fine Aggregate are weakly negatively correlated with concrete strength.

#### Multicollinearity:

The highest multicollinearity is the correlation between **water and superplasticizer**, which has a value of -0.66. This indicates they carry overlapping and opposite information. There is also moderate correlation among **cement**, **blast furnace slag**, and **fly ash** because these may act as partial substitutes in the concrete mix, so they tend to move in opposite directions. 

In [ ]:
VIFs = [(predictor, variance_inflation_factor(X,_)) \
        for _,predictor in enumerate(list(feature_columns))] 
print('Variance Inflation Factors')
for tup in VIFs:
    print('{:20}'.format(tup[0]), '{:.3f}'.format(tup[1]))

The variance inflation factors are larger for cement, blast furnace slag, fly ash, water, coarse aggregate, fine aggregate. This also indicates multi-collinearity of the factors mention above.

As mentioned in the lecture content of Machine Learning and Data Analytics(MLDA), VIF equal to 1 shows the absence of collinearity Usually, there is collinearity when VIF is larger than 5.

### Machine Learning Methods:
This project aims to predict concrete compressive strength which is a problem with a quantitative response. According to the lecture content in the fundamental part of MLDA, this task is a **regression task**. In the following part, linear regression, ridge regression, lasso regression, one decision-tree based method are applied. 

### Cross validation:

#### K-fold cross validation:

For supervised machine learning, the goal is to develop a model that can perform accurately not only on the data it was trained on, but also on previously unseen data. To evaluate a model's generalization ability, the **K-fold** method is utilized as the cross validation method.

In [ ]:
# define the kfold, shuffle the training and test dataset every time
kf = KFold(n_splits=10, random_state = 0, shuffle = True)

##### Scorers for the model

In [ ]:
# Create MSE scorer 
mse_scorer = make_scorer(mean_squared_error)

# Create r2 scorer 
r2 = make_scorer(r2_score)

#### Linear Regression:

##### Statsmodel, outliers and high leverege points:

In [ ]:
#Statsmodel fitting, it is used to show the p-values
X_stat = sm.add_constant(concrete_df.iloc[:,0:-1])
y_stat = concrete_df[label_column]

statmodel = sm.OLS(y_stat,X_stat)
estimate = statmodel.fit()

print(estimate.summary())

According to the result of the fitted model, there is a relationship between the response of concrete compressive strength and **cement, blast furnace slag, fly ash, water, superplasticizer and age**, because they have low p-values, which is smaller than 0.05.

In [ ]:
# Obtain the residuals, studentized residuals and the leverages
fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

In [ ]:
# Calculate thresholds
n = len(fitted_values)
p = X.shape[1] - 1  # exclude constant
leverage_thresh = (p + 1) / n

# Convert to arrays
studentized_residuals = np.asarray(studentized_residuals)
leverages = np.asarray(leverages)

# Identify outlier indices
outlier_indices = np.where(np.abs(studentized_residuals) > 3)[0]
high_leverage_indices = np.where(leverages > leverage_thresh)[0]
outliers = []
for idx in outlier_indices:
    outliers.append(idx)
print("Outliers are:",outliers)

# Find common indices (both outlier and high leverage)
joint_outliers = np.intersect1d(outlier_indices, high_leverage_indices)
high_leverage_and_outliers = [] 

for idx in joint_outliers:
    high_leverage_and_outliers.append(idx)
print("Comments on joint outliers (in both ax2 and ax3):",high_leverage_and_outliers)


# Plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))

# 1. Residuals plot
ax1.scatter(fitted_values, residuals, facecolors='none', edgecolors='b')
ax1.set_xlabel('Fitted values')
ax1.set_ylabel('Residuals')

# 2. Studentized Residuals plot
ax2.scatter(fitted_values, studentized_residuals, facecolors='none', edgecolors='b')
ax2.axhline(y=3, color='r', linestyle='--', linewidth=1)
ax2.axhline(y=-3, color='r', linestyle='--', linewidth=1)
# Mark joint outliers
for idx in joint_outliers:
    ax2.scatter(fitted_values[idx], studentized_residuals[idx], color='red')
    ax2.annotate(idx, (fitted_values[idx], studentized_residuals[idx]), color='red', fontsize=8)
ax2.set_xlabel('Fitted values')
ax2.set_ylabel('Studentized residuals')

# 3. Leverage vs Studentized Residuals
ax3.scatter(leverages, studentized_residuals, facecolors='none', edgecolors='b')
ax3.axhline(y=3, color='r', linestyle='--', linewidth=1)
ax3.axhline(y=-3, color='r', linestyle='--', linewidth=1)
ax3.axvline(x=leverage_thresh, color='g', linestyle='--', linewidth=1)
# Mark joint outliers
for idx in joint_outliers:
    ax3.scatter(leverages[idx], studentized_residuals[idx], color='red')
    ax3.annotate(idx, (leverages[idx], studentized_residuals[idx]), color='red', fontsize=8)
ax3.set_xlabel('Leverage')
ax3.set_ylabel('Studentized residuals')

plt.tight_layout()
plt.show()

The residuals display a clear pattern, which suggests **non-linearity** in the data. The studentized residuals indicate the presence of two outliers. The leverage plot also reveals several high-leverage points—defined as points with leverage greater than $(p+1)/n=9/1030\approx 0.009$. Notably, both outliers are also high-leverage points. However, since the dataset does not clearly define criteria for excluding data points, all observations, including the outliers, are retained for the subsequent analysis.

##### Scikit-Model:

In [ ]:
model = LinearRegression()

mse_scores_lin = cross_val_score(model, X, y, scoring=mse_scorer, cv=kf)

r2_scores = cross_val_score(model, X, y, scoring='r2', cv=kf)

print("\nAverage MSE across folds:", np.mean(mse_scores_lin))    

print("\nAverage R2 across folds:", np.mean(r2_scores))

#### Ridge regression:

In [ ]:
scoring = {'mse':'neg_mean_squared_error','R2':'r2'}

# define the model
model = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree=2),
    GridSearchCV(
        estimator = Ridge(),
        cv = kf,
        scoring = scoring,
        refit = 'mse',
        
        # param_grid determines the parameters to test (alpha is lambda in the Ridge estimator)
        # np.logspace(-3, 2, 50): array from 10^-3 to 10^2 in 50 steps (base default is 10, can also be something else)
        param_grid = {'alpha': np.logspace(-3, 2, 50)},
    )
)
model.fit(X, y)
#print(model[2].cv_results_) #to show the results and names

# obtain the results
lambdas = [p['alpha'] for p in model[2].cv_results_['params']]
mses = [neg_mse * -1 for neg_mse in model[2].cv_results_['mean_test_mse']]
r2 = [r2 for r2 in model[2].cv_results_['mean_test_R2']]
    
best_model = model.named_steps['gridsearchcv'].best_estimator_
best_mse = min(mses)
best_r2 = max(r2)

best_lambda_mse = lambdas[np.argmin(mses)]
best_lambda_r2 = lambdas[np.argmax(r2)]

print(f"Best alpha regarding mse:{best_lambda_mse:.2f}, with MSE: {best_mse:.2f}")
print(f"Best alpha regarding r2:{best_lambda_r2:.2f}, with r2: {best_r2:.2f}")
model

In [ ]:
# plot the reults
fig, ax = plt.subplots(figsize = (12, 8))
ax.plot(lambdas, mses)
ax.set_xscale('log')
ax.set_title(r"MSE as a function of $\lambda$")
ax.set_xlabel(r"Hyper-parameter $\lambda$")
ax.set_ylabel("Estimated MSE");
plt.show()

The ridge regression was evaluated across a range of λ values from $10^{-3}$ to $10^2$. The validation MSE decreased with small increases in λ due to reduced overfitting, reaching a minimum at λ = 0.45. Beyond this point, further increases in λ caused the model to underfit the data, resulting in higher MSEs.
Thus, the optimal λ value was selected as 0.45, where the model achieved the lowest MSE of 59.34. 

In [ ]:
lambdas = np.logspace(-3, 6, 50)
coefficients = list()

# for each lambda define and fit the model and save the obtained parameters in the coefficients list
for lam in lambdas:
    model = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree = 2),
        Ridge(alpha = lam)
    )
    model.fit(X, y)
    coefficients.append(model[2].coef_)
    
    
# plot the results    
coefficients = np.array(coefficients).T
fig, ax = plt.subplots(figsize = (12, 10))

for coef_vals in coefficients:
    ax.plot(lambdas, coef_vals)

ax.set_xlabel(r"Hyper-parameter $\lambda$")
ax.set_ylabel(r"Value of the model coefficients $\beta_i$")
ax.set_xscale('log');
plt.show()

#### Polynomial Regression:

Since there is non-linearity in the data, we use the polynomial regression to train the dataset.

In [ ]:
# Store results
results_mse = {}
results_r2 = {}

for feature in feature_columns:
    X_p = concrete_df[[feature]].values
    #Store MSE,R2
    mse_poly = []
    r2_poly = []
    for degree in range(1, 6):  # Degrees 1 to 5
                # Create polynomial features
                poly = PolynomialFeatures(degree=degree)
                X_poly = poly.fit_transform(X_p)
        
                # Create linear regression model
                model = LinearRegression()
        
                # Perform shuffled 10-fold CV and get negative MSE (sklearn returns negative MSE by default)
                mse_scores = cross_val_score(model, X_poly, y, scoring=mse_scorer, cv=kf)
        
                # Store average MSE
                mse_poly.append(np.mean(mse_scores))

                # Store average R2
                r2_poly.append(np.mean(r2_scores))    
        # Store MSEs for this feature
    results_mse[feature] = mse_poly
    results_r2[feature] = r2_poly
    # Plot results

plt.figure(figsize=(12, 6))
best_comb = {}
best_comb = {}

# Plot MSE
for feature, mse_list in results_mse.items():
    plt.plot(range(1, 6), mse_list, marker='o', label=feature)
    
    # Sort MSEs with their degree indices
    sorted_mse = sorted((mse, i+1) for i, mse in enumerate(mse_list))  # (MSE, degree)
    
    min_mse, best_degree = sorted_mse[0]
    best_comb[feature] = best_degree
    
    print(f"{feature}:")
    print(f"  Lowest MSE = {min_mse:.2f} at degree {best_degree}")
print(best_comb)
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Polynomial Regression MSE (Each Feature Individually)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


##### Polynomial Degree Selection Summary and Justification:

In order to model the relationship between the features and the concrete compressive strength, polynomial regression up to degree 5 is applied for each feature. The mean squared error (MSE) was computed using 10-fold cross-validation to assess the predictive performance at each polynomial degree.

While some features achieved their lowest MSE at higher degrees, the improvements over lower degrees were often marginal. Since high-degree polynomial models tend to overfit—capturing noise instead of the underlying patter. Both the lowest and second-lowest MSEs are considered for each feature to guide model complexity choices.

Below is a summary of the best MSEs and their associated degrees:

| Feature            | Lowest MSE | Degree | 
| ------------------ | ---------- | ------ | 
| Cement             | 232.98     | 1      | 
| Blast Furnace Slag | 303.89     | 5      | 
| Fly Ash            | 317.01     | 4      | 
| Water              | 248.95     | 4      | 
| Superplasticizer   | 272.21     | 1      |
| Coarse Aggregate   | 296.42     | 2      | 
| Fine Aggregate     | 293.30     | 1      | 
| Age                | 206.15     | 4      | 


### Re-sampling and Cross-Validation


We use <span style="color:red">resampling methods</span> to estimate the variability of a linear regression fit:
1. Draw $k$ samples from our training dataset
2. Fit a linear regression for each sample
3. Analyze how much the fits differ

Methods: 
1. Cross-validation:
    - model assessment: validate the performance of a method
    - model selection: validate model flexibility
2. Bootstrap: used to measure accuracy of the parameter estimate of accuracy of a method

To obtain a good trade-off between bias and variance during our cross-validation we opt for the k-fold CV method. The leave-one-out cross-validation (LOOCV) has a higher variance than k-fold CV ($k < n$ where $n$ is the number of total observations).
The validation set approach which is necessary for CV overestimates the test error but k-fold CV provides an intermediate level of bias. 
Training is done on:
$(k-1)*n/k$ 
observations.
We choose to have no overlap between the k-fold training-sets to minimize correlation in the outputs and therefore minimize the variance of the mean.

### Cross validation:

#### K-fold cross validation:

For supervised machine learning, the goal is to develop a model that can perform accurately not only on the data it was trained on, but also on previously unseen data. To evaluate a model's generalization ability, the **K-fold** method is utilized as the cross validation method.

In [ ]:
# define the kfold, shuffle the training and test dataset every time
kf = KFold(n_splits=10, random_state = 0, shuffle = True)

##### Scorers for the model

In [ ]:
# Create MSE scorer 
mse_scorer = make_scorer(mean_squared_error)

# Create r2 scorer 
r2 = make_scorer(r2_score)

#### We compute and plot the LOOCV MSEs for polynomial regression models of degree 1 to 5 using the water feature:

In [ ]:
from sklearn.model_selection import LeaveOneOut

degrees = [1, 2, 3, 4, 5]
loocv_mse = []

for d in degrees:
    poly = PolynomialFeatures(degree=d, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    loo = LeaveOneOut()
    # Use negative MSE, so take the negative to get positive MSE
    scores = cross_val_score(model, X_poly, y, cv=loo, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    loocv_mse.append(mse)

# Visualization
plt.figure(figsize=(6,4))
plt.plot(degrees, loocv_mse, marker='o', linestyle='-.-', color='blue')
plt.xlabel('Polynomial Degree')
plt.ylabel('LOOCV Mean Squared Error')
plt.title('LOOCV MSE for Polynomial Regression (All Features)')
plt.xticks(degrees)
plt.grid(True)
plt.show()

print("LOOCV MSEs for degrees 1-5:", loocv_mse)

#### We compute and plot the 10-fold CV MSEs for polynomial regression models of degree 1 to 5 using all features:

In [ ]:
degrees = [1, 2, 3, 4, 5]
kfold_mse = []

for d in degrees:
    poly = PolynomialFeatures(degree=d, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    # Use 10-fold cross-validation
    scores = cross_val_score(model, X_poly, y, cv=10, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    kfold_mse.append(mse)

# Visualization
plt.figure(figsize=(6,4))
plt.plot(degrees, kfold_mse, marker='o', linestyle='-.', color='k')
plt.xlabel('Polynomial Degree')
plt.ylabel('10-Fold CV Mean Squared Error')
plt.title('10-Fold CV MSE for Polynomial Regression (All Features)')
plt.xticks(degrees)
plt.grid(True)
plt.show()

print("10-Fold CV MSEs for degrees 1-5:", kfold_mse)

#### The results show a sudden increase in the MSE for both LOOCV and 10-Fold CV at polynomial degree 5. Unfortunately there is no clear indication for a single degree of polynomial that dominates the others. The results indicate that any degree from 1 through 4 is optimal with a MSE of 0.


### Validation set approach: 

#### We compare linear, quadratic and cubic regression models and their performance. We want to predict the concrete compressive strength from the water content first.

1. We randomly split the dataframe into a training and a test set:

In [ ]:
np.random.seed(0)
training = np.random.choice([False, True], size=len(X))

y_train = concrete_df['Concrete compressive strength(MPa, megapascals)'][training]
y_test = concrete_df['Concrete compressive strength(MPa, megapascals)'][~training]

2. The features with the highest positive correlation with concrete compressive strength are: cement (0.5), superplasticizer (0.37) and age (0.33). We fit a 4-dimensional model on the training set and compute the MSE with the predictions on the validation set for the age feature and then fit a 1-D linear regression model and compute the MSE in the same way for the cement and superplasticizer feature:

In [ ]:
# Split the data into training and test sets for the age feature (4-D polynomial)
X_train = sm.add_constant(np.column_stack((concrete_df['Age (day)'][training], concrete_df['Age (day)'][training] ** 2, concrete_df['Age (day)'][training] ** 3, concrete_df['Age (day)'][training] ** 4)))
X_test = sm.add_constant(np.column_stack((concrete_df['Age (day)'][~training], concrete_df['Age (day)'][~training] ** 2, concrete_df['Age (day)'][~training] ** 3, concrete_df['Age (day)'][~training] ** 4)))

# Fit the model
model = sm.OLS(y_train, X_train)
poly_results = model.fit()

y_predictions_poly = poly_results.predict(X_test)
# Calculate MSE and R2      
mse_poly = mean_squared_error(y_test, y_predictions_poly)
r2_poly = r2_score(y_test, y_predictions_poly)
print(f"Polynomial Regression MSE: {mse_poly:.4f}\n")
print(f"Polynomial Regression R2: {r2_poly:.4f}\n")

# Split the data into training and test sets for the cement feature (linear regression)
X_train = sm.add_constant(concrete_df['Cement (component 1)(kg in a m^3 mixture)'][training])
X_test = sm.add_constant(concrete_df['Cement (component 1)(kg in a m^3 mixture)'][~training])

# Fit the model
model = sm.OLS(y_train, X_train)
lin_results = model.fit()
y_predictions_lin = lin_results.predict(X_test)

# Calculate MSE and R2
mse_lin = mean_squared_error(y_test, y_predictions_lin)
r2_lin = r2_score(y_test, y_predictions_lin)
print(f"Linear Regression MSE: {mse_lin:.4f}\n")
print(f"Linear Regression R2: {r2_lin:.4f}\n")

# Split the data into training and test sets for the superplasticizer feature (linear regression)
X_train = sm.add_constant(concrete_df['Superplasticizer (component 5)(kg in a m^3 mixture)'][training])
X_test = sm.add_constant(concrete_df['Superplasticizer (component 5)(kg in a m^3 mixture)'][~training])

# Fit the model
model = sm.OLS(y_train, X_train)    
lin_results = model.fit()
y_predictions_lin = lin_results.predict(X_test)

# Calculate MSE and R2
mse_lin = mean_squared_error(y_test, y_predictions_lin)
r2_lin = r2_score(y_test, y_predictions_lin)
print(f"Linear Regression MSE: {mse_lin:.4f}\n")
print(f"Linear Regression R2: {r2_lin:.4f}\n")

# Plot the results for age, cement, and superplasticizer
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
# Plot for Age
ax1.scatter(concrete_df['Age (day)'][~training], y_test, color='blue', label='Actual', alpha=0.5)
ax1.scatter(concrete_df['Age (day)'][~training], y_predictions_poly, color='red', label='Predicted', alpha=0.5)
ax1.set_xlabel('Age (day)')
ax1.set_ylabel('Concrete Compressive Strength (MPa)')
ax1.set_title('Polynomial Regression Results (degree 4)')
# Plot for Cement
ax2.scatter(concrete_df['Cement (component 1)(kg in a m^3 mixture)'][~training], y_test, color='blue', label='Actual', alpha=0.5)
ax2.scatter(concrete_df['Cement (component 1)(kg in a m^3 mixture)'][~training], y_predictions_lin, color='red', label='Predicted', alpha=0.5)
ax2.set_xlabel('Cement (component 1)(kg in a m^3 mixture)')
ax2.set_ylabel('Concrete Compressive Strength (MPa)')
ax2.set_title('Linear Regression Results')
# Plot for Superplasticizer
ax3.scatter(concrete_df['Superplasticizer (component 5)(kg in a m^3 mixture)'][~training], y_test, color='blue', label='Actual', alpha=0.5)
ax3.scatter(concrete_df['Superplasticizer (component 5)(kg in a m^3 mixture)'][~training], y_predictions_lin, color='red', label='Predicted', alpha=0.5)
ax3.set_xlabel('Superplasticizer (component 5)(kg in a m^3 mixture)')
ax3.set_ylabel('Concrete Compressive Strength (MPa)')
ax3.set_title('Linear Regression Results')
plt.tight_layout()
plt.show()

# Create a DataFrame for the results
results_df = pd.DataFrame({
    'Feature': ['Age (day)', 'Cement (component 1)(kg in a m^3 mixture)', 'Superplasticizer (component 5)(kg in a m^3 mixture)'],
    'Model': ['Polynomial Regression (degree 4)', 'Linear Regression', 'Linear Regression'],
    'MSE': [mse_poly, mse_lin, mse_lin],
    'R2': [r2_poly, r2_lin, r2_lin]
})
print("\nResults DataFrame:")
display(results_df)

3. The feature with the highest negative correlation with the concrete compressive strength is water content. We fit a 4-dimensional regression model on the training set and compute the MSE with the predictions on the validation set (as a test set):

In [ ]:
X_train = sm.add_constant(np.column_stack((concrete_df['Water  (component 4)(kg in a m^3 mixture)'][training], concrete_df['Water  (component 4)(kg in a m^3 mixture)'][training] ** 2, concrete_df['Water  (component 4)(kg in a m^3 mixture)'][training] ** 3, concrete_df['Water  (component 4)(kg in a m^3 mixture)'][training] ** 4)))
X_test = sm.add_constant(np.column_stack((concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training], concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training] ** 2, concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training] ** 3, concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training] ** 4)))

# Fit the model
model = sm.OLS(y_train, X_train)
poly_results = model.fit()

y_predictions_poly = poly_results.predict(X_test)
# Calculate MSE and R2      
mse_poly = mean_squared_error(y_test, y_predictions_poly)
r2_poly = r2_score(y_test, y_predictions_poly)
print(f"Polynomial Regression MSE: {mse_poly:.4f}\n")
print(f"Polynomial Regression R2: {r2_poly:.4f}\n")
# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training], y_test, facecolors='none', edgecolors='b', label='Actual', alpha=0.5)
plt.scatter(concrete_df['Water  (component 4)(kg in a m^3 mixture)'][~training], y_predictions_poly, color='r', label='Predicted', alpha=0.5)
plt.xlabel('Water  (component 4)(kg in a m^3 mixture)')
plt.ylabel('Concrete Compressive Strength (MPa)')
plt.title('Polynomial Regression Results (degree 4)')
plt.legend()
plt.show()  